## sklearn multicollinearity class

- https://www.kaggle.com/code/ffisegydd/sklearn-multicollinearity-class

In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy import stats
import warnings; warnings.filterwarnings('ignore')
#plt.style.use('ggplot')
plt.style.use('seaborn-whitegrid')
%matplotlib inline

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

from statsmodels.stats.outliers_influence import variance_inflation_factor

IF 계수가 높은 열(즉, 데이터 집합 내의 다른 열과 높은 선형성을 가지므로 제거해야 할 가능성이 있는 열)을 제거하는 데 사용할 수 있는 scikit-learn 변환기 클래스를 만들었습니다.

이 클래스는 표준 scikit-learn 트랜스포머를 기반으로 하며 VIF 수치를 계산하기 위해 통계 모델 라이브러리도 사용합니다. scikit-learn 트랜스포머에 대한 정보는 여기에서 확인할 수 있으며, 통계 모델 함수에 대한 문서는 여기에서 확인할 수 있습니다.

In [5]:
X = pd.read_csv("data/sberbank/train.csv", parse_dates=["timestamp"])
y = X.pop('price_doc')
X.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,40,9,4,0,13,22,1,0,52,4
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,36,15,3,0,15,29,1,10,66,14
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,25,10,3,0,11,27,0,4,67,10
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,15,11,2,1,4,4,0,0,26,3
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,552,319,108,17,135,236,2,91,195,14


In [6]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [7]:
class ReduceVIF(BaseEstimator, TransformerMixin): # 커스텀 전처리기를 사용하기 위해서는 이 두가지를 상속 받아야 한다
    def __init__(self, thresh = 5.0, impute=True, impute_strategy = "median"):
        self.thresh = thresh
        if impute:
            self.imputer = SimpleImputer(strategy=impute_strategy)
            
            
    def fit(self, X, y = None):
        print('ReduceVIF fit')
        if hasattr(self, 'imputer'):
            self.imputer.fit(X)
        return self
    
    def transform(self, X, y = None):
        print('ReduceVIF transform')
        columns = X.columns.tolist()
        if hasattr(self, 'imputer'):
            X = pd.DataFrame(self.imputer.transform(X), columns=columns)
        return ReduceVIF.calculate_vif(X, self.thresh)
    
    @staticmethod
    def calculate_vif(X, thresh = 5.0):
        dropped = True
        while dropped:
            variables = X.columns
            dropped = False
            vif = [variance_inflation_factor(X[variables].values, X.columns.get_loc(var)) for var in X.columns]
            
            max_vif = max(vif)
            if max_vif > thresh:
                maxloc = vif.index(max_vif)
                print(f'Dropping {X.columns[maxloc]} with vif={max_vif}')
                X = X.drop([X.columns.tolist()[maxloc]], axis=1)
                dropped=True
                
        return X
    
        

In [8]:
transformer = ReduceVIF()

# 시간을 줄이기 위해 본 예제에서는 10개만 진행
X = transformer.fit_transform(X[X.columns[-10:]], y)

ReduceVIF fit
ReduceVIF transform
Dropping cafe_count_5000_price_2500 with vif=518.6704988504272
Dropping cafe_count_5000_price_1500 with vif=137.06095267601873
Dropping church_count_5000 with vif=98.48897590266591
Dropping cafe_count_5000_price_4000 with vif=90.47290517216163
Dropping leisure_count_5000 with vif=50.91242566935234
Dropping sport_count_5000 with vif=13.91287966100182
Dropping big_church_count_5000 with vif=7.277183689499079


In [9]:
X.head()

,cafe_count_5000_price_high,mosque_count_5000,market_count_5000
0,0.0,1.0,4.0
1,0.0,1.0,14.0
2,0.0,0.0,10.0
3,1.0,0.0,3.0
4,17.0,2.0,14.0


- 모든 열에 대한 VIF를 계산할 때 처음에는 정말 느립니다. 열을 더 많이 삭제하면 속도가 빨라지지만 예제에서 10개의 열만 사용한 데에는 이유가 있습니다.
- 두 개 이상의 열이 '동점'인 경우(여러 개가 있을 수 있는 경우) 가장 먼저 찾은 열부터 삭제합니다.
- 많은 결측치를 처리하는 것에 대해서 불안할 수도 있다. 이 방식을 사용할 때는 데이터를 drop하고 향후에 널 값을 추가하여 분석하는 방법을 쓸 수도 있다.